In [1]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import plotly.graph_objects as go
from sklearn.cluster import KMeans

# Set seed for reproducibility
np.random.seed(42)  # Set seed for NumPy
random.seed(42) # Set seed for random module

In [3]:
# Data import
person = pd.read_csv("https://raw.githubusercontent.com/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/metadata/person.csv")
appliances = pd.read_csv("https://raw.githubusercontent.com/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/metadata/appliance.csv")
other_appliances = pd.read_csv("https://raw.githubusercontent.com/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/metadata/other_appliance.csv")
home = pd.read_csv("https://raw.githubusercontent.com/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/metadata/home.csv")
survey = pd.read_csv("https://raw.githubusercontent.com/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/survey_data/survey_responses.csv")
energy = pd.read_csv("https://media.githubusercontent.com/media/Rezixx/Studienarbeit_Pred_Analytics/refs/heads/main/sensor_data/household_sensors_electric.csv")

In [4]:
# Group total energy consumption per home
total_consumption_per_home = energy.groupby('consumer_id', as_index=False)['total_consumption_Wh'].sum()
total_consumption_per_home['home'] = total_consumption_per_home['consumer_id'].str.extract('(\d+)').astype(int)
total_consumption_per_home = total_consumption_per_home.drop(columns=['consumer_id'])

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Felix\AppData\Local\Temp\ipykernel_27900\458620188.py:3: SyntaxWarning: invalid escape sequence '\d'
  total_consumption_per_home['home'] = total_consumption_per_home['consumer_id'].str.extract('(\d+)').astype(int)


In [5]:
# Group and calculate appliances per household
appliances = appliances[appliances['powertype'] == 'electric']
appliances = appliances.drop(columns=['appliancetype', 'applianceclass', 'appliancesubtype', 'roomid'])
appliances["amount"] = 1 * appliances["number"]
appliances.drop(columns=['number'], inplace=True)
appliances_group = appliances.groupby('homeid', as_index=False)["amount"].sum()

# Group and calculate other_appliances per household
other_appliances['num_clean'] = other_appliances['number'].str.replace(r'\D', '', regex=True)
other_appliances['num_clean'] = other_appliances['num_clean'].astype(int)
other_appliances = other_appliances[~other_appliances['appliance_name'].str.contains('gas', case=False, na=False)]
other_appliances["amount"] = 1 * other_appliances["num_clean"]
other_appliances.drop(columns=['number', 'num_clean'], inplace=True)
other_appliances_group = other_appliances.groupby('homeid', as_index=False)["amount"].sum()

# Merge the two dataframes
appliances_merged = pd.merge(appliances_group, other_appliances_group, on='homeid', how='inner')
appliances_merged['total_appliances'] = appliances_merged['amount_x'] + appliances_merged['amount_y']
appliances_merged = appliances_merged.drop(columns=['amount_x', 'amount_y'])
appliances_merged = appliances_merged.rename(columns={'homeid': 'home'})

In [6]:
# Group homes with income bands
result_grouped = home[["homeid", "income_band"]].groupby('homeid', as_index=False).sum()
result_grouped = result_grouped.rename(columns={'homeid': 'home'})

# Merge income bands with energy consumption
income_energy_home = pd.merge(result_grouped, total_consumption_per_home, on='home', how='inner')

# Change income bands to midpoint integers
income_midpoints = {
    "Missing": 0,
    "less than £10,800": 1,
    "£10,800 to £13,499": 2,
    "£13,500 to £16,199": 3,
    "£16,200 to £19,799": 4,
    "£19,800 to £23,399": 5,
    "£23,400 to £26,999": 6,
    "£27,000 to £32,399": 7,
    "£32,400 to £37,799": 8,
    "£37,800 to £43,199": 9,
    "£43,200 to £48,599": 10,
    "£48,600 to £53,999": 11,
    "£54,000 to £65,999": 12,
    "£66,000 to £77,999": 13,
    "£78,000 to £89,999": 14,
    "£90,000 or more": 15 }
income_energy_home["income_band_mid"] = income_energy_home["income_band"].map(income_midpoints)
income_energy_home = income_energy_home.drop(columns=['income_band'])

# Merge appliances with income bands and energy consumption
appliance_income_energy_home = pd.merge(appliances_merged, income_energy_home, on='home', how='inner')
appliance_income_energy_home

,home,total_appliances,total_consumption_Wh,income_band_mid
0,47,27,1187.852340,0
1,59,24,3103.291434,15
2,61,22,2136.057633,11
3,62,22,3562.373903,10
4,63,25,4879.234576,12
...,...,...,...,...
249,331,29,301.531115,12
250,332,24,160.567792,8
251,333,14,252.520378,12
252,334,37,637.891330,1


## Handle education levels in person Data Frame
| Qualification                                                           | Value |
|-------------------------------------------------------------------------|-------|
| phd                                                                     | 9     |
| degree level qualification (or equivalent), e.g. bsc, ba, msc, ma       | 8     |
| higher educational qualification below degree level                    | 7     |
| onc / national level btec                                               | 6     |
| a-levels or highers                                                     | 5     |
| gcse grade d-g or cse grade 2-5 or standard grade level 4-6              | 4     |
| o level or gcse equivalent (grade a-c) or o grade/cse equivalent (grade 1) or standard grade level 1 | 3     |
| other qualifications                                                     | 2     |
| no formal qualifications                                                 | 1     |
| unknown                                                                 | 0     |



In [7]:
# Transform education column
person['education'] = person['education'].str.strip().str.lower()
person['education'] = person['education'].fillna('unknown')

# Map qualifications to numerical values
qualification_mapping = {
    "phd": 9,  # Highest qualification
    "degree level qualification (or equivalent), e.g. bsc, ba, msc, ma": 8,
    "higher educational qualification below degree level": 7,
    "onc / national level btec": 6,
    "a-levels or highers": 5,
    "gcse grade d-g or cse grade 2-5 or standard grade level 4-6": 4,
    "o level or gcse equivalent (grade a-c) or o grade/cse equivalent (grade 1) or standard grade level 1": 3,
    "other qualifications": 2, 
    "no formal qualifications": 1,
    "unknown": 0 } # Lowest qualification
person['education_map'] = person['education'].map(qualification_mapping)
person = person.rename(columns={'homeid': 'home'})

In [8]:
# Choose highest education per homeid for representation of household
highest_education = person.groupby('home')['education_map'].max().reset_index()

# Merge with appliance_income_energy_home
appliance_income_energy_education_home = pd.merge(appliance_income_energy_home, highest_education, on='home', how='inner')

In [12]:
appliance_income_energy_education_home

,home,total_appliances,total_consumption_Wh,income_band_mid,education_map
0,47,27,1187.852340,0,8
1,59,24,3103.291434,15,8
2,61,22,2136.057633,11,5
3,62,22,3562.373903,10,8
4,63,25,4879.234576,12,8
...,...,...,...,...,...
249,331,29,301.531115,12,9
250,332,24,160.567792,8,8
251,333,14,252.520378,12,8
252,334,37,637.891330,1,9


## KMeans Clustering with Scaling

In [16]:
scaler = StandardScaler()

# Separate home and variables
home = appliance_income_energy_education_home['home']
dataset_clustering = appliance_income_energy_education_home.drop(columns=['home'])
dataset_scaled = pd.DataFrame(scaler.fit_transform(dataset_clustering), columns=dataset_clustering.columns)

# Perform KMeans Clustering iteratively
for i in range(2, 20):
    kmeans = KMeans(n_clusters=i, n_init='auto', max_iter=300, random_state=42)
    kmeans_model = kmeans.fit(dataset_scaled)
    km_clusters = kmeans.labels_

    silhouette_score_kmeans = silhouette_score(dataset_scaled, km_clusters)
    print(f"Silhouette Score for {i} clusters: {silhouette_score_kmeans}")

dataset_scaled

Silhouette Score for 2 clusters: 0.24283551338582549
Silhouette Score for 3 clusters: 0.26432280633550853
Silhouette Score for 4 clusters: 0.25833995840457347
Silhouette Score for 5 clusters: 0.2541549823025695
Silhouette Score for 6 clusters: 0.2351714948754791
Silhouette Score for 7 clusters: 0.2330702855910369
Silhouette Score for 8 clusters: 0.22838299034758033
Silhouette Score for 9 clusters: 0.23314284363249216
Silhouette Score for 10 clusters: 0.24354903799408198
Silhouette Score for 11 clusters: 0.21575054291102627
Silhouette Score for 12 clusters: 0.21530060237697402
Silhouette Score for 13 clusters: 0.2020909193950953
Silhouette Score for 14 clusters: 0.2110399165318877
Silhouette Score for 15 clusters: 0.21175150569356851
Silhouette Score for 16 clusters: 0.21773516199201334
Silhouette Score for 17 clusters: 0.20110994191520387
Silhouette Score for 18 clusters: 0.20028075480897026
Silhouette Score for 19 clusters: 0.19715839677661576


,total_appliances,total_consumption_Wh,income_band_mid,education_map
0,-0.105986,-0.677762,-2.137473,0.235131
1,-0.490564,0.561191,1.475558,0.235131
2,-0.746949,-0.064440,0.512083,-2.032837
3,-0.746949,0.858137,0.271214,0.235131
4,-0.362371,1.709914,0.752952,0.235131
...,...,...,...,...
249,0.150399,-1.251056,0.752952,0.991120
250,-0.490564,-1.342235,-0.210523,0.235131
251,-1.772490,-1.282758,0.752952,0.235131
252,1.175940,-1.033490,-1.896605,0.991120


## KMeans Clustering without Scaling

In [10]:
home = appliance_income_energy_education_home['home']
dataset_clustering = appliance_income_energy_education_home.drop(columns=['home', 'total_consumption_Wh'])

# Perform KMeans Clustering iteratively
for i in range(2, 20):
    kmeans = KMeans(n_clusters=i, n_init='auto', max_iter=300, random_state=42)
    kmeans_model = kmeans.fit(dataset_clustering)
    km_clusters = kmeans.labels_

    silhouette_score_kmeans = silhouette_score(dataset_clustering, km_clusters)
    print(f"Silhouette Score for {i} clusters: {silhouette_score_kmeans}")

Silhouette Score for 2 clusters: 0.38147271310093644
Silhouette Score for 3 clusters: 0.3529825257939955
Silhouette Score for 4 clusters: 0.30818246613751366
Silhouette Score for 5 clusters: 0.36492002103507587
Silhouette Score for 6 clusters: 0.32520425762112626
Silhouette Score for 7 clusters: 0.30370149035210675
Silhouette Score for 8 clusters: 0.2898010898736138
Silhouette Score for 9 clusters: 0.3027152568891039
Silhouette Score for 10 clusters: 0.3029956841837439
Silhouette Score for 11 clusters: 0.3223903928679306
Silhouette Score for 12 clusters: 0.3133517472330038
Silhouette Score for 13 clusters: 0.3023821260242625
Silhouette Score for 14 clusters: 0.3032546644886508
Silhouette Score for 15 clusters: 0.2986882105792635
Silhouette Score for 16 clusters: 0.3032081074272445
Silhouette Score for 17 clusters: 0.306517941100434
Silhouette Score for 18 clusters: 0.30011163313830186
Silhouette Score for 19 clusters: 0.3085924932062144


In [18]:
# Use 2 clusters for visualizations, as this is optimal value for silhouette score

# recalculate the model for two clusters
kmeans = KMeans(n_clusters=3, n_init='auto', max_iter=300, random_state=42)
kmeans_cluster_model = kmeans.fit(dataset_scaled)
km_clusters = kmeans.labels_
centroids = kmeans.cluster_centers_.T

# Plot the centroids for the clusters
centroids_df = pd.DataFrame(centroids.T, columns=dataset_scaled.columns)
clusters = [1, 2, 3]

fig = go.Figure()
for column in centroids_df.columns:
    fig.add_trace(go.Bar(x=clusters, y=centroids_df[column], name=column))

fig.update_layout(yaxis_title='Count', barmode='group', title='K-means Clustering Results')
fig.show()